# Generate Data Train type 2

generate data train second type, its data training for sliding window prediction

In [1]:
import pandas as pd
import os
import pickle
import numpy as np

In [2]:
PATH_DATA_RAW='../../data/raw/'
PATH_DATA_CLN='../../data/clean/'
PATH_MODEL='../../data/model/'

In [3]:
os.listdir(PATH_DATA_CLN)

['.ipynb_checkpoints', 'data_clean_100k.res']

In [4]:
os.listdir(PATH_MODEL)

['char_embed_25.res', 'char_embed_50.res', 'char_embed_75.res']

# Get Data

In [5]:
data_cln=pickle.load(open(PATH_DATA_CLN+'data_clean_100k.res','rb'))

In [6]:
# ' '.join(data_cln.target.tolist())
data_cln.shape

(1900000, 2)

In [7]:
all_sentence=' '.join(data_cln.target.tolist())

In [8]:
## Word Count
all_words=pd.Series(all_sentence.split(' '))

In [9]:
all_words.value_counts().head(30)

di           167504
yang         148399
dan          128388
ini           95300
untuk         62210
itu           60835
dari          59597
dengan        57507
pada          57447
akan          47321
dalam         37813
ke            33579
juga          32035
tidak         31382
jakarta       29667
bisa          29421
dia           28300
ada           27343
saat          26638
sudah         23030
tahun         21759
lebih         20556
saya          20099
menjadi       19640
indonesia     19629
tersebut      19455
kami          19379
atau          19190
satu          17636
liga          17528
dtype: int64

## Clean Sentence

Dalam bahasa indonesia tidak mengandung :
1. kata dengan 1 huruf
2. kata dengan 2 huruf konsonan or 2 huruf vocal
3. consecutive same word is not more than 2

In [15]:
def most_consecutive_same_count(x):
    list_cnt=[]
    cnt=0
    s0=x[0]
    for s in x[1:]:
        if s==s0:
            cnt+=1
            s0=s
        else:
            list_cnt.append(cnt)
            cnt=0
            s0=s
    list_cnt.append(cnt)
    
    try:
        res=np.max(list_cnt)+1
    except:
        print(x)
            
    return res

def check_word(x):
    voc=['a','i','u','e','o']
    if len(x)==1:
        return False    
    else:
        if len(x)!=2:
            if most_consecutive_same_count(x)>2:
                return False
            else:
                return True
        
        else:
#             if (x[0] not in voc)&(x[0] not in voc)&(x[1] in voc):
            if x in ['ke','di','ya','mu']:
                return True
            else:
                return False
            


In [16]:
all_sentence_cln=' '.join([i for i in all_sentence.split(' ') if check_word(i)])

## Slide sentence

In [17]:
get_all_slide_6=[]
i=0
while True :
    wind=all_sentence_cln[0+i:6+i]
    if (len(wind)>0) & (i<2500000):
        get_all_slide_6.append(wind)
    else:
        break
    i+=1

In [18]:
get_all_slide_7=[]
i=0
while True :
    wind=all_sentence_cln[0+i:7+i]
    if (len(wind)>0) & (i<2000000):
        get_all_slide_7.append(wind)
    else:
        break
    i+=1

In [19]:
get_all_slide_8=[]
i=0
while True :
    wind=all_sentence_cln[0+i:8+i]
    if (len(wind)>0) & (i<1800000):
        get_all_slide_8.append(wind)
    else:
        break
    i+=1

In [20]:
get_all_slide_9=[]
i=0
while True :
    wind=all_sentence_cln[0+i:9+i]
    if (len(wind)>0) & (i<1500000):
        get_all_slide_9.append(wind)
    else:
        break
    i+=1

In [21]:
slides=get_all_slide_6+get_all_slide_7+get_all_slide_8+get_all_slide_9
ori_len_slides=([6]*len(get_all_slide_6))+([7]*len(get_all_slide_7))+([8]*len(get_all_slide_8))+([9]*len(get_all_slide_9))
df_data_slides=pd.DataFrame({'slides':slides,'ori_len':ori_len_slides})

In [22]:
df_data_slides['slides_no_space']=df_data_slides.slides.apply(lambda x: ''.join(x.split(' ')))
df_data_slides['space_cnt']=df_data_slides.slides.apply(lambda x: len(x.strip().split(' '))-1)
df_data_slides['slides_no_space_len']=df_data_slides.slides_no_space.apply(lambda x: len(x))

In [23]:
df_data_slides.head()

,slides,ori_len,slides_no_space,space_cnt,slides_no_space_len
0,itu mu,6,itumu,1,5
1,tu mul,6,tumul,1,5
2,u mulu,6,umulu,1,5
3,mulus,6,mulus,0,5
4,mulus,6,mulus,0,5


In [24]:
df_data_slides.space_cnt.unique()

array([1, 0, 2, 3])

In [26]:
## 3 space doesnt make any sense for bahasa
# df_data_slides[df_data_slides.space_cnt==3]

In [27]:
df_data_slides_make_sense=df_data_slides[df_data_slides.space_cnt!=3]

In [28]:
df_data_slides_make_sense.slides_no_space_len.value_counts()

6    2383354
5    2159998
7    1995391
8    1020081
4     186775
9      54267
Name: slides_no_space_len, dtype: int64

In [29]:
df_data_slides_make_sense[df_data_slides_make_sense.slides_no_space_len==6].ori_len.value_counts()

7    1461006
6     462648
8     451208
9       8492
Name: ori_len, dtype: int64

In [30]:
df_data_slides_make_sense[df_data_slides_make_sense.slides_no_space_len==6].space_cnt.value_counts()

1    1469407
0     776502
2     137445
Name: space_cnt, dtype: int64

In [31]:
df_slides6=df_data_slides_make_sense[df_data_slides_make_sense.slides_no_space_len==6]

In [33]:
# df_slides6

In [34]:
pickle.dump(df_slides6, open('../../data/clean/data_train2_for_slides.res', 'wb'))